# Imports

In [ ]:
# Import libraries
import re
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import  nltk
import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.layers import Input, Embedding, LSTM, Lambda, Dropout, Bidirectional, Concatenate, merge
import keras.backend as K
from keras.optimizers import Adadelta,Adam, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
#from saimese_utils import make_embeddings,text_to_word_list, split_and_zero_padding
from saimese_utils import*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def download_stopwords():
    nltk.download('stopwords')
    stops = set(stopwords.words('english'))
    return stops

### Load Dataset

In [ ]:
#quora_data:
train_quora = pd.read_csv('/content/drive/MyDrive/Datasets/train.csv')
test_quora =  pd.read_csv('/content/drive/MyDrive/Datasets/test.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train_quora.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
test_quora.head(5)

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [ ]:
print(train_quora.shape)
print(test_quora.shape)

(404290, 6)
(3563475, 3)


In [ ]:
# Taking a slice of train_quora and and test_qoura
train_set = train_quora.iloc[:100000, :]
test_set = test_quora.iloc[:8000, :]

## Data Preprocessing

In [ ]:
import os
os.chdir('/content')

In [ ]:
# # Extracting word2vec model
# import gzip
# import shutil
# file_path = '/content/drive/MyDrive/deep learning/GoogleNews-vectors-negative300.bin.gz'
# with gzip.open(file_path, 'rb') as f_in:
#     with open('GoogleNews-vectors-negative300.bin', 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

In [ ]:
# word2vec model.
word2vec = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Datasets/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
stops = download_stopwords()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
train_set,embeddings= make_embeddings(train_set, word2vec,stops)
test_set,embeddings = make_embeddings(test_set, word2vec,stops)

In [ ]:
del word2vec

In [ ]:
K.clear_session()

In [ ]:
train_set.head(5)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10]",0
1,1,3,4,"[1, 2, 3, 12, 13, 14, 15, 16, 15, 17, 18]","[1, 19, 20, 21, 3, 22, 23, 24, 3, 13, 14, 15, ...",0
2,2,5,6,"[26, 27, 16, 28, 3, 29, 30, 31, 32, 33, 34, 35]","[26, 27, 31, 29, 36, 37, 5, 38, 39, 40]",0
3,3,7,8,"[41, 42, 16, 43, 44, 45, 26, 27, 16, 46, 47]","[48, 3, 49, 50, 51, 52, 53, 54, 51, 2, 55, 5, ...",0
4,4,9,10,"[56, 57, 58, 8, 59, 60, 61, 62, 63, 64, 65, 66]","[56, 67, 19, 68, 8, 62, 59]",0


In [ ]:
test_set.head(5)

,test_id,question1,question2
0,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 5]","[11, 12, 13, 14, 15, 16, 17, 15, 18, 19, 4, 5, 7]"
1,1,"[20, 21, 22, 23, 24, 25, 26, 27, 1, 28, 29, 30...","[1, 28, 31, 2, 23, 24, 32]"
2,2,"[33, 34, 35, 3, 36, 37, 38, 39, 40, 41, 3, 42]","[33, 43, 38, 39, 41]"
3,3,"[44, 45, 17, 46]","[33, 47, 48]"
4,4,"[1, 49, 50, 51]","[1, 52, 53, 21, 50, 51]"


### Train validation split and zero padding.

In [ ]:
# Here we took the validation size
validation_size = 5000
train_size = len(train_set) - validation_size  # after removing validation data its the new size of train data.

In [ ]:
# Defining X and Y for train_test_split
question_col = ['question1','question2']
X = train_set[question_col]   # Here it will contain questions only.
Y = train_set['is_duplicate']  # It will contain labels only.

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size,random_state = 101)

In [ ]:
print(X_train.shape)
print(Y_validation.shape)

(95000, 2)
(5000,)


In [ ]:
leaks = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
             for x1, x2 in zip(X_train.question1, X_train.question2)]
leaks = np.array(leaks)

In [ ]:
# WE need to pass our X_train data in terms of left questions and right questions. Because we have left MALSTM and right MALSTM
X_train = {'left':X_train.question1,'right':X_train.question2}
# now for validation data
X_validation = {'left':X_validation.question1,'right':X_validation.question2}

In [ ]:
# Now we convert our train data as the input that we need, its time to convert our labels to their numpy representation.
Y_train = Y_train.values
Y_validation = Y_validation.values

In [ ]:
X_train

{'left': array([[   0,    0,    0, ...,    8,   57,  372],
        [   0,    0,    0, ..., 5120, 3564, 3082],
        [   0,    0,    0, ...,  115,  290, 1785],
        ...,
        [   0,    0,    0, ...,  108, 3156,  371],
        [   0,    0,    0, ..., 8137,  108, 2260],
        [   0,    0,    0, ..., 2675, 1101, 1468]], dtype=int32),
 'right': array([[    0,     0,     0, ...,    16,    36,  4191],
        [    0,     0,     0, ...,  4803,   777,  3082],
        [    0,     0,     0, ...,  1785,  2330, 13287],
        ...,
        [    0,     0,     0, ...,    85,   671,  1923],
        [    0,     0,     0, ...,  8136,  2260,  8137],
        [    0,     0,     0, ...,  1468,  2684,   230]], dtype=int32)}

In [ ]:
max_seq_length = max(train_set.question1.map(lambda x: len(x)).max(),
                     train_set.question2.map(lambda x: len(x)).max(),
                     test_set.question1.map(lambda x: len(x)).max(),
                     test_set.question2.map(lambda x: len(x)).max())
print(max_seq_length)
# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)

212


In [ ]:
dev_idx = max(1, int(len(Y_validation) * 0.05))

In [ ]:
shuffle_indices = np.random.permutation(np.arange(len(Y_train)))
leaks_shuffled = leaks[shuffle_indices]

In [ ]:
leaks_train, leaks_val = leaks_shuffled[:-dev_idx], leaks_shuffled[-dev_idx:]

In [ ]:
leaks_train.shape

(94750, 3)

In [ ]:
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.models import Model
import tensorflow as tf


In [ ]:
# Lets initialize our parameters that we will need through-out the model.

n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
num_epochs  = 50
embedding_dim = 300

# Now define visible layers
left_input = Input(shape= (max_seq_length, ),dtype='int32')
right_input = Input(shape = (max_seq_length,), dtype = 'int32')

# Remember we have a embedding layer in  our network architecture of MaLSTM


embedding_layer = Embedding(len(embeddings),embedding_dim, weights = [embeddings], input_length = max_seq_length,trainable = False)

# Embedding verson of input

embedding_left = embedding_layer(left_input)
embedding_right = embedding_layer(right_input)



## Model

In [ ]:
lstm_layer = Bidirectional(LSTM(n_hidden, dropout= 0.17, recurrent_dropout= 0.17))
x1 = lstm_layer(embedding_left)
x2 = lstm_layer(embedding_right)

leaks_input = Input(shape=(leaks_train.shape[1],))
leaks_dense = Dense(int(50/2), activation='relu')(leaks_input)

merged = concatenate([x1, x2, leaks_dense])
merged = BatchNormalization()(merged)
merged = Dropout(0.25)(merged)
merged = Dense(50, activation= 'relu')(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.25)(merged)
preds = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=[left_input, right_input, leaks_input], outputs=preds)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 212)]        0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 212)]        0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 212, 300)     4360800     input_8[0][0]                    
                                                                 input_9[0][0]                    
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 3)]          0                                      

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience= 3)]
model.fit([X_train['left'], X_train['right'], leaks_train], Y_train, batch_size=batch_size, epochs=num_epochs,
                            validation_data=([X_validation['left'], X_validation['right'], leaks_val], Y_validation),verbose = 2,shuffle = True, callbacks= callbacks)

ValueError: ignored